In [ ]:
import sqlite3

# Connect to the database (if it doesn't exist, it will be created)
conn = sqlite3.connect('alpr.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create the 'information' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS information (
        license_number varchar(20) primary key,
        chassis_number varchar(20),
        engine_number varchar(20),
        car_company varchar(20),
        car_model varchar(20),
        year_of_manufacture INTEGER
    )
''')

# Commit the changes and close the connection
conn.commit()
print("CREATED")
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('alpr.db')
cursor = conn.cursor()

# Insert a row of values into the 'information' table
values = ('MH20EE7598', '277297279', '1245927222', 'Skoda', 'Rapid', 2023)
cursor.execute("INSERT INTO information VALUES (?, ?, ?, ?, ?, ?)", values)

# Commit the changes and close the connection
conn.commit()
print("DATA INSERTED SUCCESFULLY")
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('alpr.db')
cursor = conn.cursor()

# Delete all records from the 'information' table
cursor.execute("DELETE FROM ")

# Commit the changes
conn.commit()
print("DATA DELETED SUCCESFULLY")
# Close the connection
conn.close()


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('alpr.db')
cursor = conn.cursor()

# Retrieve data from the 'information' table
cursor.execute("SELECT * FROM information")
rows = cursor.fetchall()

# Display the retrieved data
for row in rows:
    print(row)

# Close the connection
conn.close()


In [ ]:
import sqlite3
import cv2
import pytesseract
from flask import Flask, render_template, request, jsonify
import os
import re
import webbrowser
import threading

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
app = Flask(__name__)

recognized_text = []

def extract_alphanumeric(text):
    alphanumeric = re.sub(r'\W+', '', text)
    return alphanumeric

@app.route('/', methods=['GET', 'POST'])
def index():
    global recognized_text

    if request.method == 'POST':
        file = request.files['file']
        if file:
            file_path = os.path.join("car.jpeg")
            file.save(file_path)

            img = cv2.imread(file_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            cascade = cv2.CascadeClassifier("licenseplaterecognition.xml")
            plates = cascade.detectMultiScale(gray, 1.2, 5)

            recognized_text = []

            for (x, y, w, h) in plates:
                plate_img = gray[y:y+h, x:x+w]
                plate_text = pytesseract.image_to_string(plate_img, config='--psm 6')
                plate_text = extract_alphanumeric(plate_text)
                recognized_text.append(plate_text)
                print("License Plate:", plate_text)

            return render_template('index.html', text=recognized_text)

    return render_template('index.html', text=[])

@app.route('/get-details')
def get_details():
    conn = sqlite3.connect('alpr.db')
    cursor = conn.cursor()

    license_numbers = recognized_text
   
    details = []
    for license_number in license_numbers:
        query = "SELECT * FROM information WHERE license_number LIKE ?"
        plate_text = f'%{license_number}%'
        cursor.execute(query, (plate_text,))
        rows = cursor.fetchall()

        for row in rows:
            detail = {
                'license_number': row[0],
                'chassis_number': row[1],
                'engine_number': row[2],
                'car_company': row[3],
                'car_model': row[4],
                'year of manufacture': row[5]
            }
            details.append(detail)

    conn.close()

    if not details:
        return jsonify({'message': 'it is fake plate Since there are  No matching records found for the recognized license plate.'})
    else:
        return jsonify(details)

def open_browser():
    webbrowser.open('http://localhost:5000')

if __name__ == '__main__':
    threading.Timer(1, open_browser).start()
    app.run()